### Libraries:

In [1]:
!pip install sklearn-genetic-opt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
import time

from sklearn import preprocessing

from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, Trials

from sklearn_genetic import GASearchCV
from sklearn.model_selection import KFold
from sklearn_genetic.space import Categorical, Integer, Continuous

### Warnings:

In [3]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The objective has been evaluated at this point before.")

### Read dataset:

In [ ]:
# df = pd.read_csv('data2_filled.csv')
# df

In [4]:
from google.colab import files
uploaded = files.upload()

Saving data2_filled.csv to data2_filled (1).csv


In [5]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['data2_filled.csv']))
df

,Study_Folder,Subject_Folder,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271,0.124451,0.166400
1,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382,0.124451,0.166400
2,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633,0.124451,0.166400
3,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340,0.124451,0.166400
4,1a701efe-2b88-4be4-ae73-6307162d5873,videos,0,4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772,0.124451,0.166400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60420,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,139,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763,0.125000,0.166667
60421,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,140,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050,0.125000,0.166667
60422,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,141,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886,0.125000,0.165266
60423,74e3ed3f-a2f2-4048-ad5e-7dbe390c0fd0,videos,7998,142,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049,0.124349,0.165266


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60425 entries, 0 to 60424
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Study_Folder    60425 non-null  object 
 1   Subject_Folder  60425 non-null  object 
 2   Stimuli         60425 non-null  int64  
 3   Frame Number    60425 non-null  int64  
 4   Face_Y          60425 non-null  float64
 5   Face_X          60425 non-null  float64
 6   Face_W          60425 non-null  float64
 7   Face_H          60425 non-null  float64
 8   Right_Eye_X     60425 non-null  float64
 9   Right_Eye_Y     60425 non-null  float64
 10  Left_Eye_X      60425 non-null  float64
 11  Left_Eye_Y      60425 non-null  float64
 12  Face_Angle      60425 non-null  float64
 13  X_Predictions   60425 non-null  float64
 14  Y_Predictions   60425 non-null  float64
dtypes: float64(11), int64(2), object(2)
memory usage: 6.9+ MB


### Drop columns:

In [7]:
df.drop(['Study_Folder','Subject_Folder'], axis = 1, inplace = True)

In [8]:
df

,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,0,0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271,0.124451,0.166400
1,0,1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382,0.124451,0.166400
2,0,2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633,0.124451,0.166400
3,0,3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340,0.124451,0.166400
4,0,4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772,0.124451,0.166400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60420,7998,139,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763,0.125000,0.166667
60421,7998,140,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050,0.125000,0.166667
60422,7998,141,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886,0.125000,0.165266
60423,7998,142,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049,0.124349,0.165266


In [9]:
# Checking NaN values
df.isna().sum()

Stimuli          0
Frame Number     0
Face_Y           0
Face_X           0
Face_W           0
Face_H           0
Right_Eye_X      0
Right_Eye_Y      0
Left_Eye_X       0
Left_Eye_Y       0
Face_Angle       0
X_Predictions    0
Y_Predictions    0
dtype: int64

### Define Stimuli, X,Y:

In [10]:
df.columns

Index(['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W', 'Face_H',
       'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle',
       'X_Predictions', 'Y_Predictions'],
      dtype='object')

In [11]:
features_columns = ['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W','Face_H',
                    'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle']

target_columns=['X_Predictions', 'Y_Predictions']

In [12]:
X=df[features_columns]

In [13]:
y=df[target_columns]

In [14]:
X_train_valid= X[X['Stimuli']<= 11]

In [15]:
X_test= X[X['Stimuli']> 11]

In [16]:
y_train_valid= y[X['Stimuli']<= 11]

In [17]:
y_test= y[X['Stimuli']> 11]

### Split data:

In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.3,stratify =X_train_valid['Stimuli'] ,random_state=42)

In [19]:
print("Whole dataset :",len(df))
print("X Train size", len(X_train))
print("y Train size", len(y_train))
print("X Valid size", len(X_valid))
print("y Valid size", len(y_valid))
print("X Test size", len(X_test))
print("y Test size", len(y_test))

Whole dataset : 60425
X Train size 23720
y Train size 23720
X Valid size 10167
y Valid size 10167
X Test size 26538
y Test size 26538


In [20]:
print("X Train shape", X_train.shape)
print("y Train shape", y_train.shape)
print("X Valid shape", X_valid.shape)
print("y Valid shape", y_valid.shape)
print("X Test shape", X_test.shape)
print("y Test shape", y_test.shape)

X Train shape (23720, 11)
y Train shape (23720, 2)
X Valid shape (10167, 11)
y Valid shape (10167, 2)
X Test shape (26538, 11)
y Test shape (26538, 2)


In [21]:
X_train['Stimuli'].value_counts().sort_values()/len(X_train), X_valid['Stimuli'].value_counts().sort_values()/len(X_valid)

(0     0.080017
 9     0.083347
 5     0.083474
 2     0.083474
 4     0.083516
 1     0.083600
 10    0.083642
 3     0.083642
 6     0.083685
 7     0.083769
 8     0.083853
 11    0.083980
 Name: Stimuli, dtype: float64,
 0     0.079965
 9     0.083309
 5     0.083505
 2     0.083505
 4     0.083505
 6     0.083604
 10    0.083604
 1     0.083604
 3     0.083702
 7     0.083801
 8     0.083899
 11    0.083997
 Name: Stimuli, dtype: float64)

### Drop columns:

In [22]:
drop_column = ['Stimuli','Frame Number']

In [23]:
X.drop(columns = drop_column, inplace=True)

<ipython-input-23-02a2a226598b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns = drop_column, inplace=True)


In [24]:
X

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
0,214.0,145.0,226.0,226.0,224.0,367.0,229.0,288.0,0.440271
1,208.0,137.0,234.0,234.0,226.0,370.0,230.0,288.0,0.351382
2,211.0,138.0,230.0,230.0,222.0,370.0,227.0,290.0,0.397633
3,212.0,136.0,230.0,230.0,225.0,367.0,227.0,288.0,0.493340
4,213.0,141.0,229.0,229.0,223.0,371.0,232.0,288.0,0.450772
...,...,...,...,...,...,...,...,...,...
60420,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763
60421,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050
60422,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886
60423,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049


In [25]:
X_train.drop(columns = drop_column, inplace=True)

In [26]:
X_train

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
30626,199.0,183.0,205.0,205.0,266.0,340.0,266.0,259.0,0.353743
10603,225.0,187.0,202.0,202.0,260.0,366.0,259.0,286.0,0.469339
28769,110.0,309.0,199.0,199.0,389.0,242.0,391.0,172.0,0.156968
30149,133.0,242.0,206.0,206.0,331.0,271.0,330.0,202.0,0.206556
16125,239.0,160.0,201.0,201.0,238.0,382.0,231.0,310.0,0.550115
...,...,...,...,...,...,...,...,...,...
28521,153.0,195.0,269.0,269.0,302.0,332.0,306.0,237.0,0.230310
17344,148.0,275.0,209.0,209.0,359.0,295.0,345.0,214.0,0.164995
30525,235.0,160.0,225.0,225.0,255.0,393.0,254.0,311.0,0.642546
26050,254.0,163.0,249.0,249.0,262.0,424.0,268.0,337.0,0.501909


In [27]:
X_valid.drop(columns = drop_column, inplace=True)

In [28]:
X_valid

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
127,217.0,146.0,231.0,231.0,230.0,380.0,237.0,296.0,0.381793
32452,228.0,255.0,179.0,179.0,328.0,347.0,328.0,288.0,0.379762
19084,223.0,158.0,210.0,210.0,232.0,367.0,229.0,287.0,0.608490
16347,245.0,155.0,221.0,221.0,249.0,397.0,246.0,320.0,0.688683
16544,195.0,176.0,215.0,215.0,262.0,342.0,264.0,260.0,0.333173
...,...,...,...,...,...,...,...,...,...
22209,197.0,180.0,210.0,210.0,264.0,337.0,264.0,257.0,0.321964
16792,272.0,122.0,261.0,261.0,225.0,454.0,222.0,363.0,1.236665
19581,271.0,120.0,262.0,262.0,226.0,442.0,224.0,358.0,1.236923
9269,174.0,65.0,283.0,283.0,191.0,355.0,187.0,246.0,0.364568


In [29]:
X_test.drop(columns = drop_column, inplace=True)

<ipython-input-29-7ac6106f35b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns = drop_column, inplace=True)


In [30]:
X_test

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
33887,211.0,159.0,236.0,236.0,244.0,369.0,249.0,281.0,0.362482
33888,211.0,157.0,238.0,238.0,243.0,367.0,249.0,283.0,0.379243
33889,210.0,159.0,235.0,235.0,244.0,369.0,249.0,280.0,0.351363
33890,210.0,158.0,236.0,236.0,244.0,368.0,248.0,282.0,0.357945
33891,209.0,158.0,238.0,238.0,244.0,368.0,249.0,282.0,0.360192
...,...,...,...,...,...,...,...,...,...
60420,308.0,106.0,227.0,227.0,194.0,456.0,197.0,378.0,2.428763
60421,307.0,105.0,229.0,229.0,198.0,456.0,201.0,379.0,2.077050
60422,309.0,107.0,225.0,225.0,194.0,456.0,199.0,386.0,1.936886
60423,310.0,107.0,224.0,224.0,197.0,463.0,199.0,381.0,2.330049


### MinMaxScaler:

In [31]:
minmax_scaler = preprocessing.MinMaxScaler()
X_train_scal = minmax_scaler.fit_transform(X_train)
X_valid_scal = minmax_scaler.transform(X_valid)
X_test_scal = minmax_scaler.transform(X_test)

In [32]:
X_train_scal

array([[0.36533958, 0.40566038, 0.23668639, ..., 0.38970588, 0.39111111,
        0.00370046],
       [0.42622951, 0.41509434, 0.21893491, ..., 0.37254902, 0.45111111,
        0.00538576],
       [0.15690867, 0.70283019, 0.20118343, ..., 0.69607843, 0.19777778,
        0.00083163],
       ...,
       [0.44964871, 0.35141509, 0.35502959, ..., 0.36029412, 0.50666667,
        0.00791098],
       [0.4941452 , 0.35849057, 0.49704142, ..., 0.39460784, 0.56444444,
        0.00586059],
       [0.264637  , 0.01415094, 0.6035503 , ..., 0.02205882, 0.34666667,
        0.00992458]])

In [33]:
X_valid_scal

array([[0.40749415, 0.31839623, 0.39053254, ..., 0.31862745, 0.47333333,
        0.0041094 ],
       [0.43325527, 0.5754717 , 0.08284024, ..., 0.54166667, 0.45555556,
        0.00407979],
       [0.42154567, 0.34669811, 0.26627219, ..., 0.29901961, 0.45333333,
        0.00741446],
       ...,
       [0.53395785, 0.25707547, 0.5739645 , ..., 0.28676471, 0.61111111,
        0.01657654],
       [0.30679157, 0.12735849, 0.69822485, ..., 0.19607843, 0.36222222,
        0.00385827],
       [0.36299766, 0.45518868, 0.10059172, ..., 0.41176471, 0.39777778,
        0.00418931]])

In [34]:
X_test_scal

array([[0.39344262, 0.3490566 , 0.42011834, ..., 0.34803922, 0.44      ,
        0.00382786],
       [0.39344262, 0.34433962, 0.43195266, ..., 0.34803922, 0.44444444,
        0.00407222],
       [0.3911007 , 0.3490566 , 0.41420118, ..., 0.34803922, 0.43777778,
        0.00366576],
       ...,
       [0.62295082, 0.22641509, 0.35502959, ..., 0.2254902 , 0.67333333,
        0.02678145],
       [0.62529274, 0.22641509, 0.34911243, ..., 0.2254902 , 0.66222222,
        0.03251346],
       [0.6206089 , 0.22641509, 0.35502959, ..., 0.22303922, 0.67111111,
        0.03118879]])

### Metrics:

In [35]:
metric_dict = {"MAE":mean_absolute_error, "MSE":mean_squared_error, "R2": r2_score}

## RandomizedSearchCV:

### RandomForestRegressor:

In [36]:
log_table=[]

In [37]:
start = time.time()
reg=RandomForestRegressor()
grid_dict = {'n_estimators': [50, 100, 200],
             "criterion":['squared_error'],
             'max_depth': [5,10,20,100],
             'min_samples_split': [2,4,6,8,10],
             "random_state":[1023,42,3]}

RF_rscv = RandomizedSearchCV(estimator=reg,
                             param_distributions = grid_dict,
                             n_jobs =-1,
                             refit=True,
                             verbose=5)

RF_rscv.fit(X_train_scal,y_train)
y_valid_pred=RF_rscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['RF', 'RandomizedSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


### DecisionTreeRegressor:

In [38]:
start = time.time()
reg=DecisionTreeRegressor()

grid_dict = {"criterion":['squared_error'],
             "splitter":['best', 'random'],
             "max_depth": [5,10,20,100],
             "min_samples_split": [2,4,6,8,10],
             "random_state":[1023,42,3],
             "max_features" : ['auto','sqrt','log2']}

DT_rscv = RandomizedSearchCV(estimator=reg,
                             param_distributions = grid_dict,
                             n_jobs =-1,
                             refit=True,
                             verbose=5)

DT_rscv.fit(X_train_scal,y_train)
y_valid_pred=DT_rscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['DT', 'RandomizedSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### KNeighborsRegressor:

In [39]:
start = time.time()
reg=KNeighborsRegressor()
grid_dict = {"n_neighbors": [1,3,4,5,10,20],
             "weights":['uniform','distance'],
             "algorithm":['auto','ball_tree','kd_tree','brute']}

Knn_rscv = RandomizedSearchCV(estimator=reg,
                              param_distributions = grid_dict,
                              n_jobs =-1,
                              refit=True,
                              verbose=5)

Knn_rscv.fit(X_train_scal,y_train)
y_valid_pred=Knn_rscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['KNN', 'RandomizedSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


# GridSearchCV:

### RandomForestRegressor:

In [40]:
start = time.time()
reg=RandomForestRegressor()
grid_dict = {'n_estimators': [50, 100, 200],
             "criterion":['squared_error'],
             'max_depth': [5,10,20,100],
             'min_samples_split': [2,4,6,8,10],
             "random_state":[1023,42,3]}

RF_gscv = GridSearchCV(estimator=reg,
                       param_grid = grid_dict,
                       n_jobs =-1,
                       refit=True,
                       verbose=5)

RF_gscv.fit(X_train_scal,y_train)
y_pred=RF_gscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['RF', 'GridSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 180 candidates, totalling 900 fits


### DecisionTreeRegressor:

In [41]:
start = time.time()
reg=DecisionTreeRegressor()

grid_dict = {"criterion":['squared_error'],
             "splitter":['best', 'random'],
             "max_depth": [5,10,20,100],
             "min_samples_split": [2,4,6,8,10],
             "random_state":[1023,42,3],
             "max_features" : ['auto','sqrt','log2']}

DT_gscv = GridSearchCV(estimator=reg,
                       param_grid = grid_dict,
                       n_jobs =-1,
                       refit=True,
                       verbose=5)

DT_gscv.fit(X_train_scal,y_train)
y_pred=DT_gscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['DT', 'GridSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### KNeighborsRegressor:

In [42]:
start = time.time()
reg=KNeighborsRegressor()
grid_dict = {"n_neighbors": [1,3,4,5,10,20],
             "weights":['uniform','distance'],
             "algorithm":['auto','ball_tree','kd_tree','brute']}

Knn_gscv = GridSearchCV(estimator=reg,
                        param_grid = grid_dict,
                        n_jobs =-1,
                        refit=True,
                        verbose=5)

Knn_gscv.fit(X_train_scal,y_train)
y_pred=Knn_gscv.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['KNN', 'GridSearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


# BAYESIAN OPTIMIZATION WITH hyperopt package:

### RandomForestRegressor:

In [43]:
space = {"n_estimators": hp.quniform('n_estimators', 50, 200, 1),
         'max_depth': hp.quniform('max_depth', 5, 100, 1),
         'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
         "random_state": hp.choice('random_state',(1023,42,3))}

# Define the objective function
def objective(params):
    reg = RandomForestRegressor( n_estimators=int(params['n_estimators']),
                                 max_depth=int(params['max_depth']),
                                 min_samples_split=int(params['min_samples_split']),
                                 random_state=int(params['random_state']))
    score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': -score, 'status': 'ok'}

# Define the search algorithm and search space
trials = Trials()
algo = tpe.suggest

# Run the optimization
start = time.time()
best = fmin(objective, space, algo=algo, trials=trials, max_evals=5,return_argmin=False)

# Train a model with the selected hyperparameters
RF_bopt = RandomForestRegressor( n_estimators=int(best['n_estimators']),
                             max_depth=int(best['max_depth']),
                             min_samples_split=int(best['min_samples_split']),
                             random_state=int(best['random_state']))

RF_bopt.fit(X_train_scal, y_train)
y_valid_pred=RF_bopt.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['RF', 'BAYESIAN OPTIMIZATION', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

100%|██████████| 5/5 [15:19<00:00, 183.85s/trial, best loss: 0.05351568967663617]


### DecisionTreeRegressor:

In [44]:
space = {'max_depth': hp.quniform('max_depth', 5, 100, 1),
         'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
         "random_state": hp.choice('random_state',(1023,42,3)),
         "splitter" : hp.choice("splitter",['best', 'random']),
         "max_features" : hp.choice("max_features",['auto','sqrt','log2'])}

# Define the objective function
def objective(params):
    reg = DecisionTreeRegressor( max_depth=int(params['max_depth']),
                                 min_samples_split=int(params['min_samples_split']),
                                 random_state=int(params['random_state']),
                                 splitter=str(params['splitter']),
                                 max_features=str(params['max_features']))

    score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': -score, 'status': 'ok'}

# Define the search algorithm and search space
trials = Trials()
algo = tpe.suggest

# Run the optimization
start = time.time()
best = fmin(objective, space, algo=algo, trials=trials, max_evals=5,return_argmin=False)

# Train a model with the selected hyperparameters
DT_bopt = DecisionTreeRegressor( max_depth=int(best['max_depth']),
                             min_samples_split=int(best['min_samples_split']),
                             random_state=int(best['random_state']),
                             splitter=str(best['splitter']),
                             max_features=str(best['max_features']))

DT_bopt.fit(X_train_scal, y_train)
y_valid_pred=DT_bopt.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['DT', 'BAYESIAN OPTIMIZATION', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

 40%|████      | 2/5 [00:00<00:00,  3.06trial/s, best loss: 0.07304144883619178]

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(



 60%|██████    | 3/5 [00:00<00:00,  3.23trial/s, best loss: 0.07140010033162913]

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(



 80%|████████  | 4/5 [00:01<00:00,  3.13trial/s, best loss: 0.07140010033162913]

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(



100%|██████████| 5/5 [00:01<00:00,  2.86trial/s, best loss: 0.07140010033162913]

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### KNeighborsRegressor:

In [45]:
space = {"n_neighbors": hp.quniform('n_neighbors', 1, 20, 1),
         "weights": hp.choice("weights",['uniform','distance']),
         "algorithm":hp.choice("algorithm",['auto','ball_tree','kd_tree','brute'])}

# Define the objective function
def objective(params):
    reg = KNeighborsRegressor(n_neighbors=int(params['n_neighbors']),
                              weights=str(params['weights']),
                              algorithm=str(params['algorithm']),)

    score = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error').mean()
    return {'loss': -score, 'status': 'ok'}

# Define the search algorithm and search space
trials = Trials()
algo = tpe.suggest

# Run the optimization
start = time.time()
best = fmin(objective, space, algo=algo, trials=trials, max_evals=5,return_argmin=False)

# Train a model with the selected hyperparameters
Knn_bopt = KNeighborsRegressor(n_neighbors=int(best['n_neighbors']),
                                weights=str(best['weights']),
                                algorithm=str(best['algorithm']),)

Knn_bopt.fit(X_train_scal, y_train)
y_valid_pred=Knn_bopt.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['KNN', 'BAYESIAN OPTIMIZATION', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

100%|██████████| 5/5 [00:13<00:00,  2.80s/trial, best loss: 0.056859384876635985]


# GENETIC ALGORITHM OPTIMIZATION:

In [46]:
! pip install sklearn-genetic-opt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### RandomForestRegressor:

In [47]:
reg = RandomForestRegressor()

cv=KFold(n_splits=5, shuffle=True)

search_space = {'n_estimators': Integer(5,200),
                'max_depth':  Integer(5,100),
                'min_samples_split':  Integer(2,10),
                'random_state': Categorical([1023,42,3])}

start = time.time()
RF_ga = GASearchCV(estimator=reg,
                       cv=cv,
                       scoring='r2',
                       param_grid=search_space,
                       n_jobs=-1,
                       verbose=True,
                       population_size=10,
                       generations=3)

RF_ga.fit(X_train_scal, y_train)
y_valid_pred=RF_ga.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['RF', 'GASearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	10    	0.79812	0.179348   	0.871362   	0.261239   
1  	20    	0.865255	0.00949675 	0.871362   	0.83803    
2  	20    	0.86867 	0.00345201 	0.873267   	0.864014   
3  	20    	0.87097 	0.00119097 	0.873267   	0.868322   


### DecisionTreeRegressor:

In [48]:
reg = DecisionTreeRegressor()

cv=KFold(n_splits=5, shuffle=True)

search_space = {'max_depth':  Integer(5,100),
                'min_samples_split':  Integer(2,10),
                'random_state': Categorical([1023,42,3]),
                'splitter' : Categorical(['best', 'random']),
                'max_features' : Categorical(['auto','sqrt','log2'])}

start = time.time()
DT_ga = GASearchCV(estimator=reg,
                       cv=cv,
                       scoring='r2',
                       param_grid=search_space,
                       n_jobs=-1,
                       verbose=True,
                       population_size=10,
                       generations=3)

DT_ga.fit(X_train_scal, y_train)
y_valid_pred=DT_ga.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['DT', 'GASearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.676542	0.109508   	0.758327   	0.44597    
1  	20    	0.736222	0.0145293  	0.758327   	0.714351   
2  	20    	0.746867	0.0101726  	0.758327   	0.720665   
3  	20    	0.747512	0.00805644 	0.7576     	0.733641   


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(


### KNeighborsRegressor:

In [49]:
reg = KNeighborsRegressor()

cv=KFold(n_splits=5, shuffle=True)

search_space = {"n_neighbors": Integer(1, 20),
                "weights": Categorical(['uniform','distance']),
                "algorithm":Categorical(['auto','ball_tree','kd_tree','brute'])}

start = time.time()
Knn_ga = GASearchCV(estimator=reg,
                       cv=cv,
                       scoring='r2',
                       param_grid=search_space,
                       n_jobs=-1,
                       verbose=True,
                       population_size=10,
                       generations=3)

Knn_ga.fit(X_train_scal, y_train)
y_valid_pred=Knn_ga.predict(X_valid_scal)
end = time.time()
for metric in metric_dict:
    current_metric = metric_dict[metric]
    log_table.append(['KNN', 'GASearchCV', metric, metric_dict[metric](y_valid, y_valid_pred),(end-start)])

gen	nevals	fitness	fitness_std	fitness_max	fitness_min
0  	10    	0.80521	0.0119019  	0.822219   	0.780641   
1  	20    	0.809496	0.00601798 	0.817225   	0.799702   
2  	20    	0.813161	0.00869665 	0.824899   	0.802749   
3  	20    	0.818485	0.00535735 	0.826968   	0.809329   


# Results:

In [50]:
log_table = pd.DataFrame(log_table, columns = ['Model','Hyperparameter Algorithm ','Metric','Metric score','Elapsed time'])

In [51]:
log_table[log_table['Metric score']==log_table['Metric score'].max()]

,Model,Hyperparameter Algorithm,Metric,Metric score,Elapsed time
2,RF,RandomizedSearchCV,R2,0.884309,402.78506


### BEST MODEL:

In [52]:
best_model=DT_ga.best_estimator_
best_model

DecisionTreeRegressor(max_depth=80, max_features='auto', min_samples_split=7,
                      random_state=1023)

In [53]:
y_pred=best_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


In [54]:
y_pred

array([[0.125     , 0.49946638],
       [0.125     , 0.49946638],
       [0.125     , 0.49946638],
       ...,
       [0.125     , 0.49946638],
       [0.125     , 0.49946638],
       [0.125     , 0.49946638]])

In [55]:
for metric in metric_dict:
    current_metric = metric_dict[metric]
    print([metric, metric_dict[metric](y_test, y_pred)])

['MAE', 0.31812061927859525]
['MSE', 0.1526343927765636]
['R2', -1.0180474473744368]


### Save csv file with results:

In [56]:
log_table.to_csv('log_table2.csv', index=False)

In [57]:
from google.colab import files

files.download('log_table2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>